In [36]:
import pandas as pd
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
import march_madness_games as mmg

In [1]:
teams = pd.read_csv("datasets/kaggle_data/Teams.csv")
seeds = pd.read_csv("datasets/kaggle_data/TourneySeeds.csv")
slots = pd.read_csv("datasets/kaggle_data/TourneySlots.csv")
tourney_data = pd.read_csv("datasets/kaggle_data/TourneyCompactResults.csv")
regular_data = pd.read_csv("datasets/kaggle_data/RegularSeasonCompactResults.csv")


stationary = pd.read_csv("datasets/our_data/stationary", index_col =0)
avg_points_against =  pd.read_csv("datasets/our_data/avg_points_against", index_col =0)
avg_points_for = pd.read_csv("datasets/our_data/avg_points_for", index_col =0)
away_wins = pd.read_csv("datasets/our_data/away_wins", index_col =0)
bad_losses = pd.read_csv("datasets/our_data/bad_losses", index_col =0)
consistency= pd.read_csv("datasets/our_data/consistency", index_col =0)
dominance= pd.read_csv("datasets/our_data/dominance", index_col =0)
good_wins_matrix= pd.read_csv("datasets/our_data/good_wins_matrix", index_col =0)
rpi= pd.read_csv("datasets/our_data/rpi", index_col =0)
tough_wins= pd.read_csv("datasets/our_data/tough_wins", index_col =0)
win_percentage= pd.read_csv("datasets/our_data/win_percentage", index_col =0)
win_percentage_vs_tourney_teams_matrix= pd.read_csv("datasets/our_data/win_percentage_vs_tourney_teams_matrix", index_col =0)
wins_vs_tourney_teams= pd.read_csv("datasets/our_data/wins_vs_tourney_teams", index_col =0)

tourney_arr = mmg.filter_into_seasons(tourney_data)
regular_arr = mmg.filter_into_seasons(regular_data)
seeds_arr = mmg.filter_into_seasons(seeds)
slots_arr = mmg.filter_into_seasons(slots)

column_names = ["min_index_id", "max_index_id", "wins_vs_tourney_teams dif", "stationary dif"]

predictor_dfs = [wins_vs_tourney_teams, stationary]

window = 3
test_yr = 2004

def train_test_split(window, test_yr):
    x_train, y_train = mmg.generate_multiple_years_of_games(range(test_yr - window, test_yr),
                                     seeds_arr, 
                                     slots_arr, 
                                     tourney_arr, 
                                     column_names, 
                                     predictor_dfs
                                     )

    x_test, y_test = mmg.generate_multiple_years_of_games([test_yr],
                                     seeds_arr, 
                                     slots_arr, 
                                     tourney_arr, 
                                     column_names, 
                                     predictor_dfs
                                     )

    return x_train, y_train, x_test, y_test

In [33]:
x_train, y_train, x_test, y_test = train_test_split(3, 2008)

In [75]:
def cross_val_c(window):
    col_names = ['0.0001', '0.001', '0.01', '.1', '1', '10', '100']
    test_yr_range = range(2003 + window, 2016)
    scores = pd.DataFrame(index=test_yr_range, columns = col_names)
    
    c_vals = range(-4, 3)

    for yr in test_yr_range:
        x_train, y_train, x_test, y_test = train_test_split(window, yr)
        scaler = StandardScaler().fit(x_train.iloc[:, 2:])
        for c in c_vals:
            model = LogReg(C = 10**c)
            model.fit(scaler.transform(x_train.iloc[:, 2:]), y_train.values.T[0])
            scores.ix[yr, c+2] = model.score(scaler.transform(x_test.iloc[:, 2:]), y_test.values.T[0])
    return scores

In [81]:
scores_w2 = cross_val_c(2)
scores_w3 = cross_val_c(3)
scores_w4 = cross_val_c(4)
scores_w5 = cross_val_c(5)

In [82]:
scores_w2.mean()

0.0001    0.712843
0.001     0.712843
0.01      0.705628
.1        0.705628
1         0.708514
10        0.712843
100       0.714286
dtype: float64

In [83]:
scores_w3.mean()

0.0001    0.711111
0.001     0.707937
0.01      0.704762
.1        0.703175
1         0.703175
10        0.706349
100       0.706349
dtype: float64

In [84]:
scores_w4.mean()

0.0001    0.723104
0.001     0.710758
0.01      0.716049
.1        0.721340
1         0.719577
10        0.712522
100       0.712522
dtype: float64

In [85]:
scores_w5.mean()

0.0001    0.712302
0.001     0.708333
0.01      0.698413
.1        0.700397
1         0.698413
10        0.706349
100       0.708333
dtype: float64